In [1]:
from collections import defaultdict
from collections import namedtuple
import pandas as pd
import json
import time

In [2]:
t0 = time.time()

In [3]:
feat = pd.read_table("clean/features.txt", sep=' ')
feat.head()

,chr,start,end,strand,type
0,1.0,335,649,+,gene
1,1.0,538,792,+,gene
2,1.0,1807,2169,-,gene
3,1.0,2480,2707,+,gene
4,1.0,7013,9049,-,gene


Defining a namedtuple here that will be used for the features. Adding an antisense boolean attribute to differentiate between sense and antisense promotion.

In [4]:
Feat = namedtuple('Feat', ['chr', 'start', 'end', 'strand', 'type', 'antisense'], defaults = (False,))

Create a dictionary containing every position of each feature. Key is (chromosome, position, strand) and value is a list of namedtuples Feat.

In [5]:
pos = defaultdict(list)
for row in feat.itertuples(index=False):
    for i in range(row.start, row.end+1):
        item = Feat(row.chr, row.start, row.end, row.strand, row.type)
        pos[(row.chr, i, row.strand)].append(item)

In [6]:
net = pd.read_table("clean/net.txt", sep=' ')
net.head()

,pos,value,chr,reads,strand
0,156,0.481988,1,1,+
1,5793,0.481988,1,1,+
2,5871,0.481988,1,1,+
3,5883,0.481988,1,1,+
4,5889,0.481988,1,1,+


First step is counting reads for each feature. If there is an overlap, I do a weighted split where the weight is the inverse of distance from the start site.

In [7]:
d = defaultdict(float)
for n in net.itertuples():
    # making the key
    tup = (n.chr, n.pos, n.strand)
    # check if the position is part of a feature
    if tup in pos:
        keys = pos.get(tup)
        total = 0 # this is the denominator
        checksum = 0
        # on the + strand, check distance from start
        if n.strand == "+":
            # first find the denominator
            for key in keys:
                total += 1/(tup[1]-key.start + 1)                
            # populate d with the weighted reads    
            for key in keys:
                d[key] += n.reads * (1/(tup[1]-key.start + 1)) / total
                checksum += n.reads * (1/(tup[1]-key.start + 1)) / total
        # on the - strand, check distance from end    
        else:
            # first find the denominator
            for key in keys:
                total += 1/(key.end - tup[1] + 1)
            # populate d with the weighted reads    
            for key in keys:
                d[key] += n.reads * (1/(key.end - tup[1] + 1)) / total
                checksum += n.reads * (1/(key.end - tup[1] + 1)) / total
        # check that we assigned all reads somewhere
        if n.reads != round(checksum):
            print(n, "failed checksum")
    # when position is not a part of any feature
    else:
        key = Feat(n.chr, n.pos, n.pos, n.strand, 'unannotated')
        d[key] = n.reads

In [8]:
expand = defaultdict(int)
for row in d:
    for i in range(row.start, row.end+1):
        expand[(row.chr, i, row.strand)] += d[row]

In [9]:
df = pd.DataFrame([expand])
df = df.transpose()
df = df.rename(columns={0: "reads"})
df = df.reset_index(names = "og_index")
df = pd.concat([pd.DataFrame(df["og_index"].tolist(), columns = ["chr", "pos", "strand"]), df[["reads"]]], axis=1) 
df.head()

,chr,pos,strand,reads
0,1.0,156,+,1.0
1,1.0,5793,+,1.0
2,1.0,5871,+,1.0
3,1.0,5883,+,1.0
4,1.0,5889,+,1.0


In [10]:
df.to_csv("test/net_processed.txt", index=False)

In [11]:
t1 = time.time()
t1-t0

778.1535000801086